In [6]:
import numpy as np
import matplotlib.pyplot as plt
from gravity3d import compute_gravity

nx, ny, nz = 30, 30, 15
Lx, Ly, Lz = 20e3, 20e3, 4e3 

x = np.linspace(-Lx/2, Lx/2, nx)
y = np.linspace(-Ly/2, Ly/2, ny)
z = np.linspace(0, Lz, nz) 

dx, dy, dz = x[1] - x[0], y[1] - y[0], z[1] - z[0]
x_edges = np.concatenate([x - dx/2, [x[-1] + dx/2]])
y_edges = np.concatenate([y - dy/2, [y[-1] + dy/2]])
z_edges = np.concatenate([z - dz/2, [z[-1] + dz/2]])

X, Y = np.meshgrid(x, y, indexing='ij')
Zc = z + dz / 2

depth_max = 3000 
x0, y0 = 0.0, 0.0
sigma_x, sigma_y = 5e3, 5e3

Z_basin = depth_max * np.exp(-((X - x0)**2 / (2 * sigma_x**2) + (Y - y0)**2 / (2 * sigma_y**2)))

rho_model = np.zeros((nx, ny, nz))
density_contrast = -500

for k in range(nz):
    mask = z[k] < Z_basin 
    rho_model[:, :, k][mask] = density_contrast

Xobs, Yobs = np.meshgrid(x, y, indexing='ij')
Zobs = np.zeros_like(Xobs) 

gz_obs = compute_gravity(Xobs, Yobs, Zobs, x_edges, y_edges, z_edges, rho_model)


In [ ]:
import numpy as np
from scipy.interpolate import RectBivariateSpline
from scipy.optimize import differential_evolution
from gravity3d import compute_gravity

nx_ctrl, ny_ctrl = 6, 6 
x_ctrl = np.linspace(x[0], x[-1], nx_ctrl)
y_ctrl = np.linspace(y[0], y[-1], ny_ctrl)
X_ctrl, Y_ctrl = np.meshgrid(x_ctrl, y_ctrl, indexing='ij')

depth_min, depth_max = 1000, 4000
bounds = [(depth_min, depth_max)] * (nx_ctrl * ny_ctrl)

def interpolate_surface(ctrl_depths):
    ctrl_depths = np.array(ctrl_depths).reshape((nx_ctrl, ny_ctrl))
    spline = RectBivariateSpline(x_ctrl, y_ctrl, ctrl_depths, kx=3, ky=3)
    Z_spline = spline(x, y, grid=True)  
    return Z_spline

def misfit(ctrl_depths):
    Z_basin = interpolate_surface(ctrl_depths)
    rho_trial = np.zeros((nx, ny, nz))
    for k in range(nz):
        mask = z[k] < Z_basin
        rho_trial[:, :, k][mask] = density_contrast 
    gz_pred = compute_gravity(Xobs, Yobs, Zobs, x_edges, y_edges, z_edges, rho_trial)
    return np.linalg.norm(gz_pred - gz_obs)

result = differential_evolution(misfit, bounds, strategy='best1bin', maxiter=100, popsize=15, 
                                mutation=(0.5, 1), recombination=0.7, polish=True, disp=True)

recovered_ctrl_depths = result.x.reshape((nx_ctrl, ny_ctrl))
recovered_surface = interpolate_surface(recovered_ctrl_depths)
